# NOTEBOOK PARTE 2: TRANSFORMER FINETUNING TRADUCTOR PT -> ES

# CONFIGURACIÓN DEL ENTORNO

## Librerías

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"   # Filtra mensajes INFO y WARNING de TF
import random
import torch
from tqdm.auto import tqdm
import numpy as np
import matplotlib.pyplot as plt

from datasets import load_from_disk, DatasetDict
from transformers import (
    set_seed,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from peft import get_peft_config, get_peft_model, LoraConfig
import sacrebleu

## Reproducibilidad

In [2]:
SEED = 333
set_seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

## Selección dispositivo

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)

Dispositivo: cuda


## Path del corpus tokenizado

In [4]:
DATA_DIR      = "./data"
TOKENIZED_DIR = os.path.join(DATA_DIR, f"tokenized_facebook_m2m100_418M_len128")
assert os.path.isdir(TOKENIZED_DIR), f"No existe el directorio tokenizado: {TOKENIZED_DIR}"
print("Tokenized data directory:", TOKENIZED_DIR)

Tokenized data directory: ./data\tokenized_facebook_m2m100_418M_len128


# Carga del corpus tokenizado

In [5]:
# Cargar tokenizer
MODEL_NAME = "facebook/m2m100_418M"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.src_lang = "pt"
tokenizer.tgt_lang = "es"

tokenized_datasets = load_from_disk(TOKENIZED_DIR)

# — Verificaciones antes de muestreo —
print("Splits disponibles:", list(tokenized_datasets.keys()))
print(f"Tamaño train original: {len(tokenized_datasets['train'])}")
print(f"Tamaño test            : {len(tokenized_datasets['test'])}")

# — Muestreo para reducir el tiempo de entrenamiento
train_full   = tokenized_datasets["train"]
train_sample = train_full.shuffle(seed=SEED).select(range(40_000))
tokenized_datasets["train"] = train_sample

test_full    = tokenized_datasets["test"]
test_sample  = test_full.shuffle(seed=SEED).select(range(10_000))
tokenized_datasets["test"]  = test_sample

print(f"Tamaño train reducido : {len(tokenized_datasets['train'])}")
print(f"Tamaño test reducido  : {len(tokenized_datasets['test'])}")

Splits disponibles: ['train', 'test']
Tamaño train original: 1984064
Tamaño test            : 496017
Tamaño train reducido : 40000
Tamaño test reducido  : 10000


# Selección y configuración del modelo pre-entrenado

Como modelo pre entrenado se seleccionó facebook/m2m100_418M, un sistema Multilingual Machine Translation de Facebook que soporta de forma nativa la traducción directa de portugués a español. Su arquitectura Seq2Seq basada en Transformers, pre-entrenada sobre cientos de pares de idiomas, aporta un conocimiento lingüístico muy sólido desde el arranque, lo que se traduce en mejores resultados y convergencia más rápida, desarrollandose en un solo paso. Para adaptar eficientemente este “foundation model” al dominio específico del Parlamento Europeo, se empleó PEFT (Parameter-Efficient Fine-Tuning), esta es una metodología que inyecta un pequeño conjunto de parámetros adicionales (para este caso LoRA) sin tener que actualizar todos los millones de pesos originales. Con esto, se busca reducir drásticamente el uso de memoria y el tiempo de entrenamiento, intentando mantener la precisión del modelo completo

In [6]:
# Cargar modelo
# MODEL_NAME = "facebook/m2m100_418M" # ya cargado
base_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
base_model.to(device)

M2M100ForConditionalGeneration(
  (model): M2M100Model(
    (shared): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
    (encoder): M2M100Encoder(
      (embed_tokens): M2M100ScaledWordEmbedding(128112, 1024, padding_idx=1)
      (embed_positions): M2M100SinusoidalPositionalEmbedding()
      (layers): ModuleList(
        (0-11): 12 x M2M100EncoderLayer(
          (self_attn): M2M100SdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): ReLU()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
   

In [7]:
# Configuración de LoRA (PEFT)
lora_config = LoraConfig(
    r=8,                       # rango bajo para la factorización de la matriz
    lora_alpha=32,             # escala de aprendizaje relativa
    target_modules=["q_proj", "v_proj"],  # módulos a adaptar
    lora_dropout=0.05,         # dropout para regularización
    bias="none",               # no ajustar bias
    task_type="SEQ_2_SEQ_LM"   # tarea Seq2Seq
)


In [8]:
# Envolver el modelo con PEFT-LoRA
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()  # para verificar cuántos parámetros se afinan

trainable params: 1,179,648 || all params: 485,085,184 || trainable%: 0.2432


# Definición de Hiperparámetros

In [9]:
# Directorio de salida para checkpoints y logs
OUTPUT_DIR = "./outputs/m2m100_finetuned_lora"

training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=16,     # subimos a 16
    per_device_eval_batch_size=32,      # evalúa más rápido
    gradient_accumulation_steps=1,      # sin acumulación

    do_eval=True,
    eval_steps=0,   # evalúa al final de cada época
    save_steps=0,   # guarda al final de cada época
    save_total_limit=2,

    logging_steps=500,
    num_train_epochs=2,                 # entrenar 1 épocas
    learning_rate=5e-5,
    warmup_ratio=0.1,

    predict_with_generate=True,        # desactivar durante training
    fp16=True
)

print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=0,
eval_strategy=no,
eval_use_gather_object=False,
fp16=True,
fp1

# Métricas de Evaluación

In [10]:
# Definir la función de métricas (BLEU y Perplexity)
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # Decodificar las predicciones
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Decodificar las etiquetas
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # BLEU
    bleu = sacrebleu.corpus_bleu(decoded_preds, [decoded_labels]).score
    # Perplexity (usando la pérdida retornada por trainer.evaluate())
    # Trainer ya computa 'eval_loss', así que simples:
    perplexity = np.exp(eval_preds.metrics["eval_loss"])
    return {"bleu": bleu, "perplexity": perplexity}

# Entrenamiento

In [11]:
# Instanciar el Trainer y lanzar el entrenamiento
# Crear el Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],         # tu subset de validación reducido
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Lanzar entrenamiento
trainer.train()

C:\Users\jpmon\AppData\Local\Temp\ipykernel_18108\524095280.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.58.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
500,1.493900
1000,1.392400
1500,1.373000
2000,1.345400
2500,1.391000
3000,1.352000
3500,1.336600
4000,1.340500
4500,1.337900
5000,1.344000


TrainOutput(global_step=5000, training_loss=1.3706663208007812, metrics={'train_runtime': 3594.4227, 'train_samples_per_second': 22.257, 'train_steps_per_second': 1.391, 'total_flos': 2.174352359424e+16, 'train_loss': 1.3706663208007812, 'epoch': 2.0})

## Guardar el modelo para luego ejecutar en modo inferencia

In [16]:
# Nombre del directorio de inferencia
INFER_DIR = "./outputs/m2m100_finetuned_lora_inferencia"

# Crear carpeta si no existe
os.makedirs(INFER_DIR, exist_ok=True)

# Guardar modelo y tokenizer
model.save_pretrained(INFER_DIR)
tokenizer.save_pretrained(INFER_DIR)

print(f"Modelo y tokenizer guardados en: {INFER_DIR}")

Modelo y tokenizer guardados en: ./outputs/m2m100_finetuned_lora_inferencia


# Análisis de Métricas

In [25]:
# Crear subset de evaluación ligero —
# Asumimos que `tokenized_datasets` ya está cargado
# y que tu split "test" está reducido o completo.
SAMPLE_SIZE = 1000
light_test = tokenized_datasets["test"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
print(f"Evaluando sólo {len(light_test)} ejemplos para métricas rápidas.")

Evaluando sólo 1000 ejemplos para métricas rápidas.


In [27]:
from datasets import load_dataset
#Carga del raw test (JSON HuggingFace)
raw_test = load_dataset(
    "json",
    data_files={"test": "./data/clean_corpus_pt_es.json"},
    field=None,
    split="test"
)

# Subset ligero para métricas
SAMPLE_SIZE = 1000
light_raw = raw_test.shuffle(seed=SEED).select(range(SAMPLE_SIZE))
print(f"Subconjunto raw_test para generación: {len(light_raw)} ejemplos")

# Preparar modelo/tokenizer para inferencia
INFER_DIR = "./outputs/m2m100_finetuned_lora_infer"
device    = torch.device("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_inf = AutoTokenizer.from_pretrained(
    INFER_DIR,
    local_files_only=True
)
model_inf = AutoModelForSeq2SeqLM.from_pretrained(
    INFER_DIR,
    local_files_only=True
).to(device)

tokenizer_inf.src_lang = "pt"
tokenizer_inf.tgt_lang = "es"
forced_bos = tokenizer_inf.get_lang_id("es")

# Función de generación por batch (greedy)
def generate_batch(batch):
    # batch["pt"] y batch["es"] están disponibles en light_raw
    inputs = tokenizer_inf(
        batch["pt"],
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    out = model_inf.generate(
        **inputs,
        forced_bos_token_id=forced_bos,
        num_beams=1,       # greedy decode
        max_new_tokens=60
    )
    decoded = tokenizer_inf.batch_decode(out, skip_special_tokens=True)
    return {"pred": decoded}

# Ejecutar generación sobre el subset raw
light_preds = light_raw.map(
    generate_batch,
    batched=True,
    batch_size=64,
    remove_columns=light_raw.column_names
)

# Ya tienes light_preds["pred"] con las traducciones
print(light_preds[0])
print(light_preds[1])

Generating test split: 0 examples [00:00, ? examples/s]

Subconjunto raw_test para generación: 1000 ejemplos


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['early_stopping']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not va

{'pred': 'ARTÍCULO 1, PONTO 9-B (nuevo)'}
{'pred': '-//EP//TEXT REPORT A6-2007-0494 0 NOT XML V0//PT'}


In [29]:
# Extraer referencias e hipótesis
refs = light_raw["es"]          # lista de cadenas en español
hyps = light_preds["pred"]      # lista de traducciones generadas

# Calcular BLEU
bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
print(f"BLEU en subset ({len(refs)} ej): {bleu:.2f}")

# Preparar tokenized subset para evaluar pérdida
#    (usamos el mismo submuestreo pero tokenizado)
light_tok = tokenized_datasets["test"].shuffle(seed=SEED).select(range(len(refs)))

# Ghost trainer para pérdida
ghost_trainer = Seq2SeqTrainer(
    model=model_inf,
    args=training_args,           # los mismos args de fine-tuning
    tokenizer=tokenizer_inf,
    eval_dataset=light_tok
)
pred_out = ghost_trainer.predict(light_tok)

# Extraer loss y calcular Perplexity
#    Fíjate en la clave exacta de metrics: puede ser 'test_loss' o 'eval_loss'
loss_key = "test_loss" if "test_loss" in pred_out.metrics else "eval_loss"
eval_loss = pred_out.metrics[loss_key]
perplexity = np.exp(eval_loss)
print(f"Perplexity en subset: {perplexity:.2f}")

BLEU en subset (1000 ej): 34.56


C:\Users\jpmon\AppData\Local\Temp\ipykernel_18108\711444320.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  ghost_trainer = Seq2SeqTrainer(


Perplexity en subset: 3.46


# Comentarios y conclusiones

**Rendimiento cuantitativo sólido:** Con un BLEU de 34.56 en el subset de 1,000 ejemplos, el modelo demuestra una capacidad de traducción competitiva para PT→ES, capturando con buena fidelidad léxico y estructura. Un BLEU superior a 30 suele considerarse de alta calidad en tareas de traducción automática de pares de idiomas con divergencias sintácticas.

**Perplexity baja indica fluidez:** La perplexity resultante de 3.46 sugiere que el modelo asigna probabilidades elevadas a las secuencias de referencia en español. Valores cercanos a 1 indicarían predicciones casi perfectas; aunque 3.46 no es ideal, es muy razonable para un fine-tuning de dos épocas sobre datos limitados, y apunta a que la generación es coherente y fluida.

**Trade-off tiempo vs precisión:** Al usar un subset de evaluación de solo 1,000 ejemplos con decoding greedy, se ha acelerado el cálculo de métricas a 30s para BLEU y 1 min para perplexity. Esto valida que la estrategia de muestreo ligero ofreció una estimación fiable sin requerir horas de cómputo.

En conjunto, el fine-tuning con LoRA sobre M2M100 ha logrado un traductor PT→ES eficaz y eficiente, con métricas sólidas y un flujo de trabajo adaptable al tiempo y recursos disponibles.